# Sentiment Analysis - Bert Multilingual

#### This notebook aims to process data that has been previously cleaned and provide a specific score for sentiment analysis. Once this score is generated, an accuracy analysis will be performed to check the performance of the applied model.

## Import packages

In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates
from gensim.parsing.preprocessing import remove_stopwords, strip_numeric, strip_punctuation, strip_multiple_whitespaces, strip_short
from sklearn.metrics import confusion_matrix
import os
from sklearn.metrics import classification_report, confusion_matrix

## Load model

The model used is bert-base-multilingual-uncased-sentiment from the hugging face library

In [4]:
# https://huggingface.co/nlptown/bert-base-multilingual-uncased-sentiment

# Creating the tokenizer for sentiment analysis using the specified pre-trained model
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

# Creating the model for sequence classification using the specified pre-trained model
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

# Load data

The data is load, previously cleaned in the preprocessing models. The csv that will be used is data_clean_1. This is a data set cleaned, which only contains the sentence where the player name is. It also contains 30 rows per language which has been manually labeled, to enable later accuracy performance report

The selection of this specific dataset was based on two main factors. Firstly, it aligned with the specifications and examples that the BERT-multilingual model was designed to work with. Secondly, after testing various other cleaned datasets, this particular one demonstrated better accuracy and more reliable results. Some of the other datasets were yielding all positive or all negative results, making this dataset the preferred choice for sentiment analysis.

In [5]:
# Load the CSV file into a DataFrame
df_de = pd.read_csv('../Preprocessing/data_clean/labeled-data/labeled-de_clean_1-1.csv',sep = ';')
#df_en = pd.read_csv('../Preprocessing/data_clean/en_clean_1.csv')
df_en = pd.read_csv('../Preprocessing/data_clean/labeled-data/labeled-en_clean_1-1_not101010.csv')
#df_es = pd.read_csv('../Preprocessing/data_clean/en_clean_1.csv')
df_es = pd.read_csv('../Preprocessing/data_clean/labeled-data/labeled-es_clean_1-1.csv', sep = ';')


## Sentiment Analysis Process

Create a simple function that allows the model to operate for each record in the df

In [1]:
# Definition of function to assign a sentiment score for each record of the df
def sentiment_score(data):
    if isinstance(data, str):
        tokens = tokenizer.encode(data, return_tensors='pt')
        result = model(tokens)
        return int(torch.argmax(result.logits)) + 1
    else:
        return 0

In [8]:
# Runing the funtion sentiment_score for each data set per language
df_de['sentiment'] = df_de['data'].apply(lambda x: sentiment_score(x[:512]))
df_en['sentiment'] = df_en['data'].apply(lambda x: sentiment_score(x[:512]))
df_es['sentiment'] = df_es['data'].apply(lambda x: sentiment_score(x[:512]))

## Translate scores to positive, neutral and negative

In order to compare the results with the manually labled records, the following translation into positive, neutral and negative are needed

In [10]:
# Create a map for distribution
sentiment_mapping = {1: 'negativ', 2: 'neutral', 3: 'neutral', 4: 'neutral', 5: 'positiv'}

# Replace the numbers with labels using the mapping
df_de['sentiment_label'] = df_de['sentiment'].map(sentiment_mapping)
df_en['sentiment_label'] = df_en['sentiment'].map(sentiment_mapping)
df_es['sentiment_label'] = df_es['sentiment'].map(sentiment_mapping)

## Save data

Once all sentiment scores have been added, the data needs to be stored for later analysis

In [11]:
# Concatenate the dataframes vertically
merged_df = pd.concat([df_de, df_en, df_es], ignore_index=True)

# Create the folder if it doesn't exist
folder_name = 'data'

# Define the file path for saving the CSV
file_name = 'data_sentiment_final.csv'
file_path = os.path.join(folder_name, file_name)

# Convert the dataframe to CSV and save it
merged_df.to_csv(file_path, index=False)

## Accuracy Performance Analysis

Using the manually labeled rows an accuracy performance analysis is needed. The base for this analysis is the confusion matrix. The perofrmance per language per category, taking into acount f1, precision and recall

In [12]:
# Drop rows where 'Label' is NaN or empty
df_de.dropna(subset=['Label'], inplace=True)
df_en.dropna(subset=['Label'], inplace=True)
df_es.dropna(subset=['Label'], inplace=True)

## Accuracy for exact match

This accuracy measure captures the exact match, indicating that the manually given label exactly matches the predicted-mapped label. In other words, it evaluates how many predicted labels are precisely the same as the ground-truth (manually assigned) labels.

In [13]:
# Calculate the accuracy
accuracy_de = (df_de['sentiment_label'] == df_de['Label']).mean() * 100
accuracy_en = (df_en['sentiment_label'] == df_en['Label']).mean() * 100
accuracy_es = (df_es['sentiment_label'] == df_es['Label']).mean() * 100

# Print the accuracy
print("Accuracy DE: {:.2f}%".format(accuracy_de))
print("Accuracy EN: {:.2f}%".format(accuracy_en))
print("Accuracy ES: {:.2f}%".format(accuracy_es))

Accuracy DE: 56.67%
Accuracy EN: 46.67%
Accuracy ES: 43.33%


## Classification report DE

In [14]:
# Extracting the actual sentiment labels
actual_labels = df_de['Label']

# Extracting the predicted sentiment labels
predicted_labels = df_de['sentiment_label']

# Calculating the classification report based on the actual and predicted sentiment labels
report = classification_report(actual_labels, predicted_labels)

# Displaying the classification report which includes precision, recall, F1-score
print(report)

              precision    recall  f1-score   support

     negativ       0.56      0.50      0.53        10
     neutral       0.53      0.80      0.64        10
     positiv       0.67      0.40      0.50        10

    accuracy                           0.57        30
   macro avg       0.59      0.57      0.56        30
weighted avg       0.59      0.57      0.56        30



## Classification report EN

In [21]:
# Extracting the actual sentiment labels
actual_labels = df_en['Label']

# Extracting the predicted sentiment labels
predicted_labels = df_en['sentiment_label']

# Calculating the classification report based on the actual and predicted sentiment labels
report = classification_report(actual_labels, predicted_labels)

# Displaying the classification report which includes precision, recall, F1-score
print(report)

              precision    recall  f1-score   support

     negativ       0.31      1.00      0.47         4
     neutral       0.62      0.62      0.62        13
     positiv       0.50      0.15      0.24        13

    accuracy                           0.47        30
   macro avg       0.47      0.59      0.44        30
weighted avg       0.52      0.47      0.43        30



## Classification report ES

In [22]:
# Extracting the actual sentiment labels
actual_labels = df_es['Label']

# Extracting the predicted sentiment labels
predicted_labels = df_es['sentiment_label']

# Calculating the classification report based on the actual and predicted sentiment labels
report = classification_report(actual_labels, predicted_labels)

# Displaying the classification report which includes precision, recall, F1-score
print(report)


              precision    recall  f1-score   support

     negativ       0.42      0.50      0.45        10
     neutral       0.44      0.80      0.57        10
     positiv       0.00      0.00      0.00        10

    accuracy                           0.43        30
   macro avg       0.29      0.43      0.34        30
weighted avg       0.29      0.43      0.34        30



/Users/galachaparro/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/galachaparro/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/galachaparro/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(r

## Classification report overall

In [23]:
#Concatenate the dataframes vertically
merged_df = pd.concat([df_de, df_en, df_es], ignore_index=True)

# Extracting the actual sentiment labels
actual_labels = merged_df['Label']

# Extracting the predicted sentiment labels
predicted_labels = merged_df['sentiment_label']

# Calculating the classification report based on the actual and predicted sentiment labels
report = classification_report(actual_labels, predicted_labels)

# Displaying the classification report which includes precision, recall, F1-score
print(report)

              precision    recall  f1-score   support

     negativ       0.41      0.58      0.48        24
     neutral       0.52      0.73      0.61        33
     positiv       0.60      0.18      0.28        33

    accuracy                           0.49        90
   macro avg       0.51      0.50      0.46        90
weighted avg       0.52      0.49      0.45        90



# Notebook Output

This notebook creates a CSV file that will contain the sentiment score and label predicted by the model for each row of data. Simultaneously, this provides us with information on the accuracy of the model for each language and overall performance.

# Next steps for Buyer04

As the next steps for Buyer04, we would recommend exploring other models that are specifically designed for accurate analysis in the context of sports news. Different models might offer better performance and results for sentiment analysis in this domain.

Another approach to improve accuracy is to find the most suitable way to map the sentiment scores into the labels positive, negative, and neutral. Fine-tuning this mapping process based on the specific characteristics of sports news data may lead to more precise sentiment predictions and better overall performance of the sentiment analysis system.